In [1]:
!git init
!git remote add origin https://github.com/AlexOlsen/DeepWeeds
!git pull origin master

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /kaggle/working/.git/
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 37 (delta 0), reused 2 (delta 0), pack-reused 33 (from 1)
Unpacking objects: 100% (37/37), 556.80 KiB | 5.74 MiB/s, done.
From https://github.com/AlexOlsen/DeepWeeds
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master


In [2]:
import datetime,time,os
import pandas as pd
import numpy as np
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from keras.optimizers import Adam
from keras.models import Model, load_model
from sklearn.metrics import confusion_matrix, classification_report
from keras.applications.inception_v3 import InceptionV3
from keras.applications import VGG19
from keras.utils import to_categorical
from keras.layers import Dense, GlobalAveragePooling2D
import keras
from keras.applications.inception_v3 import InceptionV3

import gc
from sklearn.model_selection import train_test_split

In [3]:
OUTPUT_DIRECTORY="./output/"
# CONSTANTS 
LABEL_DIRECTORY = "./labels/"
MODEL_DIRECTORY = "./models/"
IMG_DIRECTORY = "/kaggle/input/masked-images/Major_project"
RAW_IMG_SIZE = (224, 224)
IMG_SIZE = (224, 224)
INPUT_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], 3)
MAX_EPOCH = 100
BATCH_SIZE = 32
FOLDS = 5
STOPPING_PATIENCE = 4
LR_PATIENCE = 16
INITIAL_LR = 0.0001
CLASSES = [0, 1, 2, 3, 4, 5, 6, 7, 8]
CLASS_NAMES = ['Chinee Apple',
               'Lantana',
               'Parkinsonia',
               'Parthenium',
               'Prickly Acacia',
               'Rubber Vine',
               'Siam Weed',
               'Snake Weed',
               'Negatives']



output_directory = OUTPUT_DIRECTORY
if not os.path.exists(output_directory):
        os.makedirs(output_directory)

In [4]:
df=pd.read_csv(LABEL_DIRECTORY+"labels.csv")
least_freq=sorted(df.Species.value_counts())[0]
temp=pd.DataFrame()
for species in  set(df["Species"]):
    temp = pd.concat([temp,df[df["Species"]==species].sample(least_freq,random_state=49)])
temp.to_csv("./resampled.csv",index=False)

df=pd.read_csv("./resampled.csv")

In [5]:
df=pd.read_csv("./resampled.csv")
one_hot=pd.get_dummies(df['Species'])
df=df.join(one_hot)
df=df.drop(columns=['Label','Species'],axis=1)
df.columns


Index(['Filename', 'Chinee apple', 'Lantana', 'Negative', 'Parkinsonia',
       'Parthenium', 'Prickly acacia', 'Rubber vine', 'Siam weed',
       'Snake weed'],
      dtype='object')

In [6]:
train_dataframe,test_dataframe=train_test_split(df,random_state=49,shuffle=True,train_size=0.75)
val_dataframe,test_dataframe=train_test_split(test_dataframe,random_state=49,shuffle=True,test_size=0.5)

In [7]:
# train_dataframe['Label']=to_categorical(train_dataframe['Label']).tolist()
# val_dataframe['Label']  =to_categorical(val_dataframe['Label']).tolist()
# test_dataframe['Label'] =to_categorical(test_dataframe['Label']).tolist()

train_image_count = train_dataframe.shape[0]
val_image_count = train_dataframe.shape[0]
test_image_count = test_dataframe.shape[0]

In [8]:
y_colums=['Chinee apple', 'Lantana', 'Negative', 'Parkinsonia','Parthenium', 'Prickly acacia', 'Rubber vine', 'Siam weed','Snake weed']
train_data_generator = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="constant",
    shear_range=0.2,
    zoom_range=(0.5, 1),
    horizontal_flip=True,
    rotation_range=360,
    channel_shift_range=25,
    brightness_range=(0.75, 1.25))
# Validation image augmentation
val_data_generator = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="constant",
    shear_range=0.2,
    zoom_range=(0.5, 1),
    horizontal_flip=True,
    rotation_range=360,
    channel_shift_range=25,
    brightness_range=(0.75, 1.25))
# No testing image augmentation (except for converting pixel values to floats)
test_data_generator = ImageDataGenerator(rescale=1. / 255)
train_data_generator = train_data_generator.flow_from_dataframe(
    train_dataframe,
    IMG_DIRECTORY,
    x_col='Filename',
    y_col=y_colums,
    target_size=RAW_IMG_SIZE,
    batch_size=BATCH_SIZE,
    has_ext=True,
    class_mode='raw')
# Load validation images in batches from directory and apply rescaling
val_data_generator = val_data_generator.flow_from_dataframe(
    val_dataframe,
    IMG_DIRECTORY,
    x_col="Filename",
    y_col=y_colums,
    target_size=RAW_IMG_SIZE,
    batch_size=BATCH_SIZE,
    has_ext=True,
    class_mode='raw')
# Load test images in batches from directory and apply rescaling
test_data_generator = test_data_generator.flow_from_dataframe(
    test_dataframe,
    IMG_DIRECTORY,
    x_col="Filename",
    y_col=y_colums,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    has_ext=True,
    shuffle=False,
    class_mode='raw')

Found 1959 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 4851 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  warnings.warn(


Found 346 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 789 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  warnings.warn(


Found 352 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 784 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  warnings.warn(


In [9]:
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)

x = base_model.output
# Add a global average pooling layer
x = GlobalAveragePooling2D(name='avg_pool')(x)
# Add fully connected output layer with sigmoid activation for multi label classification
outputs = Dense(len(CLASSES), activation='softmax', name='fc9')(x)
# Assemble the modified model
model = Model(inputs=base_model.input, outputs=outputs)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# Checkpoints for training
model_checkpoint = ModelCheckpoint(output_directory + "lastbest-0.keras", verbose=1, save_best_only=True,monitor='val_loss')
early_stopping = EarlyStopping(monitor="val_loss",min_delta=0,patience=STOPPING_PATIENCE,verbose=2,mode="auto",restore_best_weights=True,start_from_epoch=10)
tensorboard = TensorBoard(log_dir=output_directory, histogram_freq=0, write_graph=True, write_images=False)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.5, patience=LR_PATIENCE, min_lr=0.000003125)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=INITIAL_LR),
              loss="categorical_crossentropy",
              metrics=['categorical_accuracy'])

checkpoint_callback = ModelCheckpoint(filepath=output_directory+'model_checkpoint.keras',
                                      monitor='val_loss',
                                      verbose=1,
                                      save_best_only=True,
                                      mode='min')
# csv_logger = CSVLogger(output_directory + "training_metrics.csv")
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

global_epoch = 0
restarts = 0
last_best_losses = []
last_best_epochs = []
# Train model until MAX_EPOCH, restarting after each early stop when learning has plateaued

# Define callbacks

while global_epoch < MAX_EPOCH:
    history = model.fit(
        train_data_generator,
        epochs=MAX_EPOCH - global_epoch,
        validation_data=val_data_generator,
        callbacks=[tensorboard, model_checkpoint, early_stopping, reduce_lr],
        shuffle=False)
    last_best_losses.append(min(history.history['val_loss']))
    last_best_local_epoch = history.history['val_loss'].index(min(history.history['val_loss']))
    last_best_epochs.append(global_epoch + last_best_local_epoch)

    if early_stopping.stopped_epoch == 0:
        print("Completed training after {} epochs.".format(MAX_EPOCH))
        break
    else:
        global_epoch = global_epoch + early_stopping.stopped_epoch - STOPPING_PATIENCE + 1
        print("Early stopping triggered after local epoch {} (global epoch {}).".format(
            early_stopping.stopped_epoch, global_epoch))
        print("Restarting from last best val_loss at local epoch {} (global epoch {}).".format(
            early_stopping.stopped_epoch - STOPPING_PATIENCE, global_epoch - STOPPING_PATIENCE))
        restarts = restarts + 1
        model.compile(loss= "categorical_crossentropy", optimizer=Adam(learning_rate=INITIAL_LR / 2 ** restarts),
                      metrics=['categorical_accuracy'])
        model_checkpoint = ModelCheckpoint(output_directory + "lastbest-{}.keras".format(restarts),
                                           monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        gc.collect()

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 692ms/step - categorical_accuracy: 0.3007 - loss: 1.5612

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_loss improved from inf to 1.29002, saving model to ./output/lastbest-0.keras
62/62 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - categorical_accuracy: 0.3016 - loss: 1.5585 - val_categorical_accuracy: 0.3988 - val_loss: 1.2900 - learning_rate: 1.0000e-04
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 507ms/step - categorical_accuracy: 0.4768 - loss: 1.1880
Epoch 2: val_loss improved from 1.29002 to 1.04458, saving model to ./output/lastbest-0.keras
62/62 ━━━━━━━━━━━━━━━━━━━━ 38s 615ms/step - categorical_accuracy: 0.4774 - loss: 1.1874 - val_categorical_accuracy: 0.5000 - val_loss: 1.0446 - learning_rate: 1.0000e-04
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step - categorical_accuracy: 0.5640 - loss: 1.0472
Epoch 3: val_loss improved from 1.04458 to 0.93173, saving model to ./output/lastbest-0.keras
62/62 ━━━━━━━━━━━━━━━━━━━━ 37s 597ms/step - categorical_accuracy: 0.5646 - loss: 1.0465 - val_categorical_accuracy: 0.6358 - val_loss: 0.9317 - learning_rate: 1.0000e-04
Epoch 4/100
62/6

In [11]:
# Save last best model info
with open(output_directory + "last_best_models.csv", 'w', newline='') as file:
     writer = csv.writer(file, delimiter=',')
     writer.writerow(['Model file', 'Global epoch', 'Validation loss'])
     for i in range(restarts + 1):
         writer.writerow(["lastbest-{}.keras".format(i), last_best_epochs[i], last_best_losses[i]])
    # Load the last best model
model = load_model(output_directory + "lastbest-{}.keras".format(last_best_losses.index(min(last_best_losses))))

In [12]:
# Evaluate model on test subset for kth fold
predictions = model.predict(test_data_generator)
y_true=np.argmax(test_data_generator.labels,axis=1)
y_pred=np.argmax(predictions, axis=1)
# y_pred[np.max(predictions, axis=1) < 1 / 9]


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 244ms/step


In [13]:
# Generate and print classification metrics and confusion matrix

print(classification_report(y_true, y_pred, labels=CLASSES, target_names=CLASS_NAMES))
report = classification_report(y_true, y_pred, labels=CLASSES, target_names=CLASS_NAMES, output_dict=True)
with open(output_directory + 'classification_report.csv', 'w') as f:
    for key in report.keys():
        f.write("%s,%s\n" % (key, report[key]))
conf_arr = confusion_matrix(y_true, y_pred, labels=CLASSES)
print(conf_arr)
np.savetxt(output_directory + "confusion_matrix.csv", conf_arr, delimiter=",")

# Clear model from GPU after each iteration
print("Finished testing")


                precision    recall  f1-score   support

  Chinee Apple       0.94      0.94      0.94        32
       Lantana       1.00      1.00      1.00         1
   Parkinsonia       0.00      0.00      0.00         1
    Parthenium       0.95      0.98      0.96        94
Prickly Acacia       0.95      0.95      0.95       120
   Rubber Vine       0.94      0.92      0.93        78
     Siam Weed       0.96      0.96      0.96        24
    Snake Weed       0.00      0.00      0.00         0
     Negatives       0.00      0.00      0.00         2

     micro avg       0.94      0.94      0.94       352
     macro avg       0.64      0.64      0.64       352
  weighted avg       0.94      0.94      0.94       352

[[ 30   0   0   0   2   0   0   0   0]
 [  0   1   0   0   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   0]
 [  0   0   0  92   0   2   0   0   0]
 [  1   0   0   1 114   2   1   0   1]
 [  0   0   0   4   2  72   0   0   0]
 [  1   0   0   0   0   0  23   0  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [14]:
!zip -r ./output.zip ./output

  adding: output/ (stored 0%)
  adding: output/lastbest-60.keras (deflated 9%)
  adding: output/lastbest-20.keras (deflated 9%)
  adding: output/lastbest-15.keras (deflated 9%)
  adding: output/lastbest-11.keras (deflated 9%)
  adding: output/train/ (stored 0%)
  adding: output/train/events.out.tfevents.1741503230.e9bded963883.31.50.v2 (deflated 90%)
  adding: output/train/events.out.tfevents.1741501642.e9bded963883.31.34.v2 (deflated 90%)
  adding: output/train/events.out.tfevents.1741509520.e9bded963883.31.120.v2 (deflated 90%)
  adding: output/train/events.out.tfevents.1741499545.e9bded963883.31.12.v2 (deflated 90%)
  adding: output/train/events.out.tfevents.1741509338.e9bded963883.31.118.v2 (deflated 90%)
  adding: output/train/events.out.tfevents.1741498326.e9bded963883.31.2.v2 (deflated 90%)
  adding: output/train/events.out.tfevents.1741499924.e9bded963883.31.16.v2 (deflated 90%)
  adding: output/train/events.out.tfevents.1741501463.e9bded963883.31.32.v2 (deflated 90%)
  adding:

In [15]:
import numpy as np
import cv2
from keras.models import load_model
from tensorflow.keras.applications.vgg19 import preprocess_input

# Load the trained model
MODEL_PATH = "/kaggle/working/output/lastbest-0.keras"
model = load_model(MODEL_PATH)

# Define class labels
CLASS_NAMES = ['Chinee Apple', 'Lantana', 'Parkinsonia', 'Parthenium', 'Prickly Acacia', 
               'Rubber Vine', 'Siam Weed', 'Snake Weed', 'Negative']  

def predict_image(image_path):
    """Loads an image, preprocesses it, and predicts the weed type."""
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize to match model input size
    IMG_SIZE = (224, 224)  # Ensure this matches the training size
    image = cv2.resize(image, IMG_SIZE)
    
    # Convert to array and preprocess
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = preprocess_input(image)  # Normalize pixel values
    
    # Make prediction
    predictions = model.predict(image)
    
    # Get the highest probability class
    predicted_class = CLASS_NAMES[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    
    print(f"Predicted Class: {predicted_class} (Confidence: {confidence:.2f}%)")
    return predicted_class, confidence

# Example Usage:
image_path = "/kaggle/input/deepweeds/images/20160928-140337-0.jpg"  # Change this to your test image
predict_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class: Lantana (Confidence: 100.00%)


('Lantana', 100.0)